In [1]:
%matplotlib inline

import re
import numpy as np
import subprocess
import os, stat
import os.path
import time
import tempfile
import shutil
import logging
import urllib
import enum

import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
logging.getLogger("").setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logging.getLogger("").addHandler(ch)

logger = logging.getLogger("main")
logger.debug("Test")

Test


In [3]:
!python --version

Python 3.6.6


In [5]:
class Architecture(enum.Enum):
    LAPTOP = 1
    DESKTOP = 2
    SUPERCOMPUTER_MET = 3
    SUPERCOMPUTER_UIO = 4
    

class SIM_TYPE(enum.Enum):
    FBL = 1
    CTCS = 2
    CDKLM = 3
    
architecture = Architecture.DESKTOP
sim_type = SIM_TYPE.CTCS

In [ ]:
#git_url = "git@github.com:babrodtk/gpu-ocean.git"
git_url = None
git_command = "git"
my_env = os.environ
shell = True

# If we are on windows, we might need to modify the path and git command:
if os.name == 'nt':
    my_env["PATH"] = "C:\\Program Files\\Git\\mingw64\\bin\\;C:\\Program Files\\Git\\usr\\bin\\;" + my_env["PATH"]
    git_command = "git.exe"
if os.name == 'posix':
    shell = False

try:
    git_url = subprocess.check_output([git_command, "rev-parse", "--show-toplevel"], stderr=subprocess.STDOUT, shell=shell)
    git_url = os.path.abspath(os.path.join(git_url.decode("utf-8").rstrip(), ".git"))
except subprocess.CalledProcessError as e:
    logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
logger.info(git_url)
    
    
benchmark_script_relpath = "gpu_ocean/demos/testCasesDemos/run_benchmark.py"
cdklm_python_class_relpath = "gpu_ocean/SWESimulators/CDKLM16.py"
benchmark_script_version = "5c2214573269367fbdb6d67ace07f2ffa57d37ba"
benchmark_script_options = ["--simulator", sim_type.name, '--steps_per_download', '10', '--iterations', '3', \
                           "--nx", "2048", "--ny", "2048"]
outfile = os.path.join(os.getcwd(), "output_" + time.strftime("%Y_%m_%d-%H_%M_%S") + ".npz")


git_versions_cdklm = [
    #(git hash, git commit message, block size laptop, block size desktop, block size supercomputer)
    ("a126bab97e690b7c642814e3c8b96f9879adf487", "original (per arch optimized block size)", (32,4), (12,12), (12, 12), (12, 12)), #Change blocksize here!
    ("5d817bb7cd2b369039117d19aae7d669a9a2e53a", "Optimized shared memory use Hm", (32,4), (12,12), (12, 12), (12, 12)),
    ("2e5da01457874ad5087398da77b9102ee991fb94", "Optimized shared memory use RHx/RHy", (32,4), (12,12), (12, 12), (12, 12)),
    ("fcd145c8c29f3d38a024685fdd0fc3cf9580366c", "Optimized shared memory use Q", (32,4), (12,12), (12, 12), (12, 12)),
    ("74e957bb41e391a5c5fbb19c3cac393079272dd3", "Optimized shared memory use F", (32,4), (12,12), (12, 12), (12, 12)),
    ("5aba525d1d64937c03e8d2b33bb7f6d80b97a81a", "Optimized shared memory use G", (32,4), (12,12), (12, 12), (12, 12)),
    ("addb061fe1cfccebb6fab70f9826be3752162b0b", "Optimized shared memory use Qy", (32,4), (12,12), (12, 12), (12, 12)),
    ("aab115045fd6d313940fa37be3149155ba4ead70", "Made variables const", (32,4), (12,12), (16, 16), (12, 12)),
    ("f6a911368b429df608eb5e1218c28bc14d6ffbe2", "Updated blocksize experiment code", (32,4), (12,12), (12, 12), (12, 12)),
    ("339eb0c9a3bf04875507fab24d8924e24718a2eb", "Changed default block size", (16,8), (24,8), (8, 12), (32, 12)), #Change blocksize here!
    ("1319edf92c20b8d5fd9fc8c0e0ed58176f9cc2ba", "Tried to reduce register use", (16,8), (24,8), (8, 12), (32, 12)),
    ("6de871265b820aecc63d68a2e1f858605eb367c7", "Refactored to optimize register use", (16,8), (24,8), (8, 12), (32, 12)),
    ("f6d1727d747b486203df91079b83a9e717668852", "Added logger to CDKLM", (16,8), (24,8), (8, 12), (32, 12)),
    ("40f9d4817fa084de7cebfd3cf6d20254b93e7393", "Added fast math compilation flag", (16,8), (24,8), (8, 12), (32, 12)),
    #("e2af2159becbd3c8769903eb4bea602a96d6c3a1", "Compilation flags experimentation", (16,8), (24,8), (8, 12), (32, 12)),
    ("12536844bdc4459dcf4cc92776faea3a81d0a32c", "New optimal block size", (32,8), (32,12), (16, 8), (32, 12)) #Change blocksize here!
]


git_versions_ctcs = [
    #(git hash, git commit message, block size laptop, block size desktop, block size supercomputer)
    ("9507b86aa57bdcedccdf3840435b31b20005dc11", "Original CUDA", (32,4), (32,4), (12, 12)), #Change blocksize here!
    ("06e92ae9da98e74e1a41e94d90081a349a3b66ad", "Using const everywhere", (32,4), (32,4), (16, 16)),
    ("b0faacf64a4be91442095153b8cec7d1efff37a1", "Reduced register count (-maxrregcount=32)", (32,4), (32,4), (16, 16)),
    
    ("0aa5f2483a3c8ad75552fd2141f8cc3ea5d67f38", "Packing wall_bc", (32,4), (32,4), (16, 16)),
    ("08eaf4636238d0553cea7d307253ef44cc380106", "using fast math compiler flag", (32,4), (32,4), (16, 16)),
    ("08eaf4636238d0553cea7d307253ef44cc380106", "Finding best block size (no new commit)", (32,4), (32,4), (16, 16)), # Find blocksize here
    
    ("22d25b210baddd14f1ffb0bdba358995eb746b43", "Using multiple streams in CTCS", (32,4), (32,4), (16, 16)),
    ("2d90aae49a63f5904897d3417a8678f1a57d3e8d", "Reduing number of blocks for BCs", (32,4), (32,4), (16, 16)),
    ("1ff345b57211f828c2e9cc9f7a879a2a909b2f62", "Removing maxrregcount flag", (32, 4), (32,4), (32,4))
]

git_versions = git_versions_cdklm
if sim_type == SIM_TYPE.CTCS:
    git_versions = git_versions_ctcs


for version, log, laptop, desktop, supercomputer in git_versions:
    logger.info(version + " - " + log)

In [ ]:
#Create temporary directory
tmpdir = tempfile.mkdtemp(prefix='git_tmp')
logger.debug("Writing to " + tmpdir)

In [ ]:
def git(cwd, options):
    a = None
    try:
        cmd = [git_command] + options
        logger.debug("Git cloning: " + cwd + " -- " + str(cmd))
        a = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=shell, cwd=cwd, env=my_env)
    except subprocess.CalledProcessError as e:
        logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
    return a.decode("utf-8") 

    
git_clone = os.path.join(tmpdir, "git_clone")
stdout = git(tmpdir, ["clone", git_url, git_clone])
logger.debug(stdout)

In [ ]:
for version, log, laptop_block, desktop_block, supercomputer_met_block, supercomputer_uio_block in git_versions:
    logger.debug("checkout " + version)
    logger.debug("    with commit msg: " + log)
    stdout = git(git_clone, ["checkout", "--force", '-q', version])
    logger.debug("stdout: " + str(stdout))
    
    logger.debug("Checkout " + benchmark_script_relpath)
    stdout = git(git_clone, ["checkout", "--force", '-q', benchmark_script_version, "--", benchmark_script_relpath])
    logger.debug("stdout: " + str(stdout))
    
    block_size_options = None
        
    if architecture == Architecture.DESKTOP or architecture == Architecture.SUPERCOMPUTER:
        
        if architecture == Architecture.LAPTOP:
            block_size_options = ["--block_width", str(laptop_block[0]), "--block_height", str(laptop_block[1])] 

            
        if architecture == Architecture.DESKTOP:
            block_size_options = ["--block_width", str(desktop_block[0]), "--block_height", str(desktop_block[1])]
            
        if architecture == Architecture.SUPERCOMPUTER_MET:
            block_size_options = ["--block_width", str(supercomputer_met_block[0]), "--block_height", str(supercomputer_met_block[1])]
            
        if architecture == Architecture.SUPERCOMPUTER_UIO:
            block_size_options = ["--block_width", str(supercomputer_uio_block[0]), "--block_height", str(supercomputer_uio_block[1])]
                    
    a = None
    try:
        #cmd = ["python", "--version"]
        cmd = ["python", os.path.join(git_clone, benchmark_script_relpath)] + benchmark_script_options + \
                ["--output", "benchmark_" + version + ".npz"] + block_size_options
        a = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=shell, cwd=tmpdir, env=my_env)
    except subprocess.CalledProcessError as e:
        logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
    logger.debug("Output:\n" + a.decode("utf-8"))
    
    # Cleaning git folder so that it can be deleted
    #stdout = git(git_clone, ["-n", ])
        
print("benchmarking finished")

In [ ]:
print("hei")

In [ ]:
versions, labels, laptop_block, desktop_block, supercomputer_met_block, supercomputer_uio_block = list(zip(*git_versions))
megacells = np.full((len(versions)), np.nan)

for i, ver in enumerate(git_versions):
    version, log, l_block, d_block, hpc_block  = ver
    filename = os.path.join(tmpdir, "benchmark_" + version + ".npz")
    with np.load(filename) as version_data:
        megacells[i] = version_data[sim_type.name]

np.savez(outfile, versions=versions, labels=labels, megacells=megacells)

In [ ]:
sim_type.name

## Plot results

In [ ]:
### NOTE: Static files
outfile_laptop = os.path.join(os.getcwd(), "laptop_cdklm_profiling.npz")
outfile_desktop = "desktop_cdklm_profiling.npz"
#outfile_supercomputer = outfile
outfile_supercomputer_met = "supercomputer_fermi_cdklm_profiling.npz"
outfile_supercomputer_uio = "supercomputer_fermi_cdklm_profiling.npz"
###

with np.load(outfile_laptop) as data:
    logger.debug("laptop file: " + outfile_laptop)
    versions = data['versions']
    labels = data['labels']
    megacells_laptop = data['megacells']
    normalized_megacells_laptop = megacells_laptop / megacells_laptop[0]
    
with np.load(outfile_desktop) as data:
    assert(np.all(versions == data['versions']))
    logger.debug("desktop file: " + outfile_desktop)
    megacells_desktop = data['megacells']
    normalized_megacells_desktop = megacells_desktop / megacells_desktop[0]
    
with np.load(outfile_supercomputer_met) as data:
    assert(np.all(versions == data['versions']))
    logger.debug("supercomputer MET file: " + outfile_supercomputer_met)
    megacells_supercomputer_met = data['megacells'] 
    normalized_megacells_supercomputer_met = megacells_supercomputer_met / megacells_supercomputer_met[0]

with np.load(outfile_supercomputer_uio) as data:
    assert(np.all(versions == data['versions']))
    logger.debug("supercomputer UiO file: " + outfile_supercomputer_uio)
    megacells_supercomputer_uio = data['megacells'] 
    normalized_megacells_supercomputer_uio = megacells_supercomputer_uio / megacells_supercomputer_uio[0]
    
fig = plt.figure()

fig, ax = plt.subplots(figsize=(12,8))

index = np.arange(len(versions))
bar_width = 0.2

ax.bar(index-1.5*bar_width, normalized_megacells_laptop, bar_width, label='Laptop')
ax.bar(index-0.5*bar_width, normalized_megacells_desktop, bar_width, label='Desktop')
ax.bar(index+0.5*bar_width, normalized_megacells_supercomputer_met, bar_width, label='Supercomputer MET')
ax.bar(index+1.5*bar_width, normalized_megacells_supercomputer_uio, bar_width, label='Supercomputer UiO')

plt.xticks(index, labels, rotation='vertical')
plt.grid()

plt.legend()

In [ ]:
def remove_readonly(func, path, _):
    "Clear the readonly bit and reattempt the removal"
    os.chmod(path, stat.S_IWRITE)
    func(path)

shutil.rmtree(tmpdir, onerror=remove_readonly)
logger.debug("Removed tempdir " + tmpdir)